<a href="https://colab.research.google.com/github/bennyqp/artificial-inspiration/blob/main/Image%20Generation/ai_image_generation_super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Image-Super-Resolution

This is a notebook for batch image-super-resolution. 

It a modified notebook. The original is from Erdene-Ochir Tuguldur.

https://github.com/tugstugi

# Install ISR

In [ ]:
!pip -q install ISR gdown

In [ ]:
pip install 'h5py<3.0.0'

## Get the pre-trained weights and data

In [ ]:
import os

if not os.path.exists('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf'):
  # download pretrained weights
  !gdown https://drive.google.com/uc?id=1Sn9FFKXNWOPXdNzy-zmyVYB4eNRnXTKD
  !gdown https://drive.google.com/uc?id=1_1GGoHF5oq3W_iVVxx1SRDIa_mjwoe26
  !gdown https://drive.google.com/uc?id=1ro0Rk4xAqo-OEDGgLIYz-F0BHvmUUT2E
  !gdown https://drive.google.com/uc?id=1o3l_I60xHkdiWZG7UM0nZBblEontHs2W
  !mkdir weights
  !mv *.hdf5 weights

## Load images which should be scaled up

You can load a zip file from google Drive here or you can upload the images yourself.

In [ ]:
import gdown

!gdown --id 1B45O5WURT3BwfbZIRNHP6Rj4Liz20BL_

& unzip

In [ ]:
zip_path = "isr_demo_portraits.zip"

!unzip {zip_path} -d input

In [ ]:
print("Number of images: " + str(len(os.listdir("input"))))

# Let's get started with ISR:

## Set input and output

In [12]:
input = "input" #Input Folder
output = "output" #Final Output Folder

Set network

In [13]:
from ISR.models import RDN, RRDN

rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')

## 1) Fourfold magnification in two steps

In [ ]:
import numpy as np
from PIL import Image
import os, sys

output1path = "step1" #Folder for intermediate step
path = input
output2path = output 

if not os.path.exists(output1path):  
    os.makedirs(output1path)
if not os.path.exists(output2path): 
    os.makedirs(output2path)

dirs = os.listdir( path )

#Remove all hidden Files
for item in dirs:
    if item.startswith('.'):
        shutil.rmtree(os.path.join(path, item))
        
dirs = os.listdir( path )

check_path = "check"
os.makedirs(check_path)
counter = 1
for item in dirs:
  if not item.startswith('.'):
    finalOutputName = os.path.join(output2path, os.path.splitext(item)[0] + "_sr" + os.path.splitext(item)[1])
    if os.path.exists(finalOutputName) == False:
  
        #Vergrößerung Schritt 1
        img = Image.open(os.path.join(path, item))
        sr_img = rdn.predict(np.array(img))
        img = Image.fromarray(sr_img)
        img.save(os.path.join(output1path, item))
        print("Image " + str(counter) + " of " + str(len(dirs)) + " Step 1 / 2")

        #Vergrößerung Schritt 2
        img2 = Image.open(os.path.join(output1path, item))
        sr_img2 = rdn.predict(np.array(img2))
        img2 = Image.fromarray(sr_img2)
        img2.save(finalOutputName)
        os.remove(os.path.join(output1path, item))
        print("Image " + str(counter) + " of " + str(len(dirs)) + " Step 2 / 2")
        
    else:
        print(item + " is already in the destination folder.")
    
    newCheckName = str(counter) + "of" + str(len(dirs))
    os.rename(check_path, newCheckName)
    check_path = newCheckName
    counter = counter + 1

print("All done.")
os.makedirs("All_done")

ifAllDone = os.listdir( output1path )
if len(ifAllDone) == 0:
    os.rmdir(output1path)

## 2) Double magnification

In [ ]:
import numpy as np
from PIL import Image
import os, sys

path = input
outputPath = output 

if not os.path.exists(outputPath):  
    os.makedirs(outputPath)

dirs = os.listdir( path )

counter = 1
for item in dirs:
  if not item.startswith('.'):
    img = Image.open(os.path.join(path,item))
    sr_img = rdn.predict(np.array(img))
    img = Image.fromarray(sr_img)
    img.save(os.path.join(outputPath,item))
    print("Image " + str(counter) + " of " + str(len(dirs)))
    counter = counter + 1

print("Done with " + str(len(outputDirs)) + " images.")

#More useful stuff

## Zip images for download

In [ ]:
import shutil

zip_name = 'isr_images'
directory_name = 'output'

shutil.make_archive(zip_name, 'zip', directory_name)

## Reduce image size

#####Define Function

In [26]:
from __future__ import print_function
def resize(inputResizePath, outputResizePath, targetWidth, targetHeight):
  resizeDirs = os.listdir( inputResizePath )
  if not os.path.exists(outputResizePath):  
    os.makedirs(outputResizePath)
  counter = 0
  for image in resizeDirs:
        im = Image.open(os.path.join(inputResizePath, image))
        im = im.resize((targetWidth,targetHeight), Image.ANTIALIAS)
        im.save(os.path.join(outputResizePath,image))
        counter = counter + 1
        print("Progress: " + str(counter) + " / " + str(len(resizeDirs)))
  print("resized " + str(counter) + " images.")

#####Reduce image size

In [ ]:
resize("input", "smallImages", 100, 100)